In [53]:
import json
import numpy
import operator

In [47]:
gt_path = '../../../data/posted_GT/gt_answer_key/'
# New
agg_gt_answ_file = '../../../data/aggregate_gt/agg_answer_key.json'
agg_metric_file = '../../../data/aggregate_gt/agg_metric_parameters.json'
sub_path = '../../../data/team_submissions/Uncharted/GroundTruth/'

In [22]:
def clean_sub(sub, index_of_ad, index_of_score):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    for entry in sub:
        # skip over aggregate single answer
        if type(entry) == list:
            if entry[index_of_score] > i:
                print "ORDERING PROBLEM"
                return None
            i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        # skip over aggregate single answer
        if type(entry) == list:
            if entry[index_of_ad] not in seen:
                uniqs.append(entry)
                seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [23]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [59]:
def AggMetric(gt_answ, sub_answ, feature):
    scale_cutoff = agg_metric_params[feature]
    diff = gt_answ - sub_answ
    if numpy.absolute(diff) <= (gt_answ * scale_cutoff):
        metric = 1
    else:
        metric = 0
        
    return metric

# SET EXTRACTIONS

In [24]:
#extractions = 'Uncharted_Uncharted_only'
extractions = 'Uncharted_Uncharted_and_lattice'
#extractions = 'Uncharted_CMU'

if extractions == 'Uncharted_Uncharted_only':
    sub_gt = sub_path + 'uncharted_extractions_GT.json'
elif extractions == 'Uncharted_Uncharted_and_lattice':
    sub_gt = sub_path + 'uncharted_lattice_extractions_GT.json'
elif extractions == 'CMU':
    sub_gt = sub_path + 'uncharted_lattice_extractions_CMU_clusters_GT.json'

## Load Submissions

In [25]:
f = open(sub_gt, 'r')
data = eval(f.read())
f.close()

pf_sub = []
id_sub = []
facet_sub = []
agg_sub = []

for entry in data:
    if entry['questionType'] == 'Point Fact':
        pf_sub.append(entry)
    elif entry['questionType'] == 'Cluster Identification':
        id_sub.append(entry)
    elif entry['questionType'] == 'Cluster Facet':
        facet_sub.append(entry)
    elif entry['questionType'] in ['MODE', 'MIN', 'MAX', 'AVG']:
        agg_sub.append(entry)
    else:
        print entry['questionType']

Submissions = {}
Submissions['Point Fact'] = pf_sub
Submissions['Cluster Identification'] = id_sub
Submissions['Cluster Facet'] = facet_sub
Submissions['Cluster Aggregate'] = agg_sub

del pf_sub
del id_sub
del facet_sub
del agg_sub

## Load GT

In [49]:
pf_path = gt_path + 'pf_quest2ads.json'
f = open(pf_path, 'r')
pf_gt = eval(f.read())
f.close()

id_path = gt_path + 'id_quest2ads.json'
f = open(id_path, 'r')
id_gt = eval(f.read())
f.close()

facet_path = gt_path + 'facet_quest2ads.json'
f = open(facet_path, 'r')
facet_gt = eval(f.read())
f.close()

agg_path = gt_path + 'agg_quest2ads.json'
f = open(agg_path, 'r')
agg_gt = eval(f.read())
f.close()

# New
f = open(agg_gt_answ_file, 'r')
agg_answ_gt = eval(f.read())
f.close()

f = open(agg_metric_file, 'r')
agg_metric_params = eval(f.read())
f.close()

Ground_Truth = {}
Ground_Truth['Point Fact'] = pf_gt
Ground_Truth['Cluster Identification'] = id_gt
Ground_Truth['Cluster Facet'] = facet_gt
Ground_Truth['Cluster Aggregate'] = agg_gt

del pf_gt
del id_gt
del facet_gt
del agg_gt

## Protocol

In [27]:
protocol = {}
protocol['Point Fact'] = {}
protocol['Point Fact']['ad_idx'] = 1
protocol['Point Fact']['score_idx'] = 2
protocol['Point Fact']['questionType'] = ['Point Fact']
protocol['Cluster Identification'] = {}
protocol['Cluster Identification']['ad_idx'] = 0
protocol['Cluster Identification']['score_idx'] = 1
protocol['Cluster Identification']['questionType'] = ['Cluster Identification']
protocol['Cluster Facet'] = {}
protocol['Cluster Facet']['ad_idx'] = 1
protocol['Cluster Facet']['score_idx'] = 2
protocol['Cluster Facet']['questionType'] = ['Cluster Facet']
protocol['Cluster Aggregate'] = {}
protocol['Cluster Aggregate']['ad_idx'] = 1
protocol['Cluster Aggregate']['score_idx'] = 2
protocol['Cluster Aggregate']['questionType'] = ['MODE', 'MIN', 'MAX', 'AVG']

In [28]:
AvgP = {}

## Create Dictionary

In [29]:
for qtype in protocol.keys():
    AvgP[qtype] = {}

    sub_data = Submissions[qtype]
    gt_data = Ground_Truth[qtype]

    try:
        del submissions
    except:
        pass
    for sub in sub_data:
        if sub['questionType'] not in protocol[qtype]['questionType']:
            print sub['questionType']
            print "TROUBLE"
            print qtype
        submissions = clean_sub(sub['answers'], protocol[qtype]['ad_idx'], protocol[qtype]['score_idx'])
        q_id = sub['question_id']
        gt = gt_data[q_id]
        AvgP[qtype][q_id] = AvgPrecision(gt, submissions, protocol[qtype]['ad_idx'], protocol[qtype]['score_idx'])

## Remove Question 94 from Aggregate

In [30]:
del AvgP['Cluster Aggregate']['94']

## Reformat Output

In [66]:
NewEval = {}

for qtype in AvgP.keys():
    NewEval[qtype] = {}
    for quest in AvgP[qtype].keys():
        NewEval[qtype][quest] = {}
        NewEval[qtype][quest]['avg_precision'] = AvgP[qtype][quest]

## Calculate Overall Aggregate Metric and Reformat Output

In [73]:
for quest in Submissions['Cluster Aggregate']:
    q_id = quest['question_id']
    # Initiate metric at 0
    metric = 0
    if q_id != '94':
        if len(quest['answers']) > 0:
            sub_answer = quest['answers'][0]
            metric = AggMetric(agg_answ_gt[q_id]['agg_answer'], sub_answer, agg_answ_gt[q_id]['feature'])
        NewEval['Cluster Aggregate'][q_id]['agg_metric'] = metric

## Calculate Point Fact Extraction Metric

## Save Avg Precision Dict

In [ ]:
save_file = extractions +'.json'
with open(save_file, 'w') as outfile:
    json.dump(AvgP, outfile)

In [33]:
Submissions['Cluster Aggregate'][0].keys()

['questionType', 'answers', 'question_id']

In [63]:
AvgP.keys()

['Cluster Identification', 'Point Fact', 'Cluster Facet', 'Cluster Aggregate']

In [74]:
NewEval['Cluster Aggregate']

{'1': {'agg_metric': 0, 'avg_precision': 0.46693381708792586},
 '10': {'agg_metric': 1, 'avg_precision': 0.8760859501266968},
 '100': {'agg_metric': 1, 'avg_precision': 1.0},
 '11': {'agg_metric': 0, 'avg_precision': 0.8825436133293068},
 '12': {'agg_metric': 0, 'avg_precision': 0.6553446922760625},
 '13': {'agg_metric': 0, 'avg_precision': 0.13890574152726246},
 '14': {'agg_metric': 0, 'avg_precision': 0.19458266496137114},
 '15': {'agg_metric': 0, 'avg_precision': 0.8421082067953298},
 '16': {'agg_metric': 1, 'avg_precision': 0.8659863945578231},
 '17': {'agg_metric': 1, 'avg_precision': 0.3193276908343853},
 '18': {'agg_metric': 1, 'avg_precision': 0.8007281968433949},
 '19': {'agg_metric': 0, 'avg_precision': 0.44540367648603724},
 '2': {'agg_metric': 0, 'avg_precision': 0.013010297496339826},
 '20': {'agg_metric': 0, 'avg_precision': 0.8052264586251813},
 '21': {'agg_metric': 0, 'avg_precision': 0.8841565896064286},
 '22': {'agg_metric': 0, 'avg_precision': 0.011253168326065557},
